# DeepEval demo en Binder (deepeval_demo.ipynb)

Este cuaderno muestra dos rutas:
1) **Sin API keys** (usando `JsonCorrectnessMetric`, no requiere LLM).
2) **Con API key** (usando métricas con LLM como `AnswerRelevancyMetric` y `GEVal`).

> **Sugerencia**: Si estás en Binder, este repo debe incluir un `requirements.txt` (ya preparado) para que el entorno se construya automáticamente.

In [14]:

# === Instalación/Comprobación de dependencias ===
import importlib, sys, subprocess

def pip_install(*packages):
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', '-q', '-U', *packages])

try:
    import deepeval  # noqa
except Exception:
    pip_install('deepeval', 'python-dotenv')

print('Entorno listo ✅')


Entorno listo ✅

In [15]:
from getpass import getpass
import os

os.environ ["OPENAI_API_KEY"] = getpass("Pega aquí tu clave API: ")

Pega aquí tu clave API:  ········


## Parte A · Evaluación **sin API keys** (JsonCorrectnessMetric)
`JsonCorrectnessMetric` valida que el `actual_output` cumpla un **esquema JSON** (Pydantic). 
No utiliza un LLM, por lo que funciona sin configurar claves.

In [16]:

from pydantic import BaseModel, RootModel
from typing import List
from deepeval import evaluate
from deepeval.test_case import LLMTestCase
from deepeval.metrics import JsonCorrectnessMetric
from openai import OpenAI


client = OpenAI()  # lee la clave de OPENAI_API_KEY 

# Definimos el esquema esperado
class Item(BaseModel):
    name: str
    price: float

class ItemList(RootModel[List[Item]]):
    pass

metric = JsonCorrectnessMetric(
    expected_schema=ItemList,
    model="gpt-4o-mini",
    include_reason=False,   # no usamos LLM para la 'reason'
    strict_mode=True        # exige coincidencia exacta de esquema
)

# Dos casos de prueba: uno correcto y otro incorrecto
ok_output = '[{"name": "Cuaderno", "price": 3.5}, {"name": "Bolígrafo", "price": 1.2}]'
ko_output = '{"name": "Cuaderno", "price": "tres"}'  # NO es lista y price no es float

test_cases = [
    LLMTestCase(input='Devuélveme una lista de artículos', actual_output=ok_output),
    LLMTestCase(input='Devuélveme una lista de artículos', actual_output=ko_output),
]

# Ejecutamos la evaluación
_ = evaluate(test_cases=test_cases, metrics=[metric])


✨ You're running DeepEval's latest Json Correctness Metric! (using gpt-4o-mini, strict=True, async_mode=True)...

======================================================================

Metrics Summary

- ✅ Json Correctness (score: 1.0, threshold: 1.0, strict: True, evaluation model: gpt-4o-mini, reason: None, 
error: None)

For test case:

...

Overall Metric Pass Rates

Json Correctness: 50.00% pass rate

======================================================================

⚠ WARNING: No hyperparameters logged.
» ]8;id=4004;https://deepeval.com/docs/evaluation-prompts\Log hyperparameters]8;;\ to attribute prompts and models to your test runs.

================================================================================

✓ Evaluation completed 🎉! (time taken: 0.06s | token cost: None)
» Test Results (2 total tests):
   » Pass Rate: 50.0% | Passed: 1 | Failed: 1

 ================================================================================ 

» Want to share evals with your team, or a place for your test cases to live? ❤️ 🏡
  » Run 'deepeval view' to analyze and save testing results on Confident AI.

## Parte B · Métricas con **LLM** (requiere API key)
Las métricas como `AnswerRelevancyMetric` o `GEVal` usan un **LLM como juez**. 
Por defecto DeepEval utiliza modelos de OpenAI; configura `OPENAI_API_KEY` (o usa otro proveedor soportado) antes de ejecutar.

In [17]:

# Ejemplo con AnswerRelevancyMetric + GEval
from deepeval.metrics import AnswerRelevancyMetric
from deepeval.test_case import LLMTestCase, LLMTestCaseParams
from deepeval import evaluate

answer_relevancy = AnswerRelevancyMetric(
    threshold=0.7,
    model='gpt-4o-mini',      # puedes cambiar a otro modelo soportado
    include_reason=True,
)
tc = LLMTestCase(
    input="What if these shoes don't fit?",
    actual_output='We offer a 30-day full refund at no extra cost.',
    expected_output="You're eligible for a free full refund within 30 days of purchase.",
)

_ = evaluate([tc], [answer_relevancy])


✨ You're running DeepEval's latest Answer Relevancy Metric! (using gpt-4o-mini, strict=False, async_mode=True)...

======================================================================

Metrics Summary

- ✅ Answer Relevancy (score: 1.0, threshold: 0.7, strict: False, evaluation model: gpt-4o-mini, reason: The 
score is 1.00 because the response directly addresses the concern about the shoes not fitting without any 
irrelevant statements., error: None)

For test case:

...

Overall Metric Pass Rates

Answer Relevancy: 100.00% pass rate

======================================================================

⚠ WARNING: No hyperparameters logged.
» ]8;id=877469;https://deepeval.com/docs/evaluation-prompts\Log hyperparameters]8;;\ to attribute prompts and models to your test runs.

================================================================================

✓ Evaluation completed 🎉! (time taken: 3.97s | token cost: 0.00021315 USD)
» Test Results (1 total tests):
   » Pass Rate: 100.0% | Passed: 1 | Failed: 0

 ================================================================================ 

» Want to share evals with your team, or a place for your test cases to live? ❤️ 🏡
  » Run 'deepeval view' to analyze and save testing results on Confident AI.

---
### Notas
- Para ejecutar métricas con LLM **sin OpenAI**, revisa la documentación de modelos soportados (Anthropic, Gemini, Ollama, etc.) y pasa `model=...` al crear la métrica.
- Este cuaderno está preparado para ejecutarse en Binder si tu repo incluye `requirements.txt`.